In [ ]:
# Header starts here.
from sympy.physics.units import *
from sympy import *

# Rounding:
import decimal
from decimal import Decimal as DX
def iso_round(obj, pv, rounding=decimal.ROUND_HALF_EVEN):
    import sympy
    """
    Rounding acc. to DIN EN ISO 80000-1:2013-08
    place value = Rundestellenwert
    """
    assert pv in set([
        # place value   #  round to:
        1,              #  1
        0.1,            #  1st digit after decimal
        0.01,           #  2nd
        0.001,          #  3rd
        0.0001,         #  4th
        0.00001,        #  5th
        0.000001,       #  6th
        0.0000001,      #  7th
        0.00000001,     #  8th
        0.000000001,    #  9th
        0.0000000001,   # 10th
        ])
    try:
        tmp = DX(str(float(obj)))
        obj = tmp.quantize(DX(str(pv)), rounding=rounding)
    except:
        for i in range(len(obj)):
            tmp = DX(str(float(obj[i])))
            obj[i] = tmp.quantize(DX(str(pv)), rounding=rounding)
    return obj

# LateX:
kwargs = {}
kwargs["mat_str"] = "bmatrix"
kwargs["mat_delim"] = ""
# kwargs["symbol_names"] = {FB: "F^{\mathsf B}", }

# Units:
(k, M, G ) = ( 10**3, 10**6, 10**9 )
(mm, cm, deg) = ( m/1000, m/100, pi/180)
Newton = kg*m/s**2
Pa     = Newton/m**2
MPa    = M*Pa
GPa    = G*Pa
kN     = k*Newton

half = S(1)/2

# Header ends here.
#
a = var("a", positive=True)
w1 = var("omega1")

w2z, w3z = var("w2z, w3z")
a1z, a2z, a3z = var("a1z, a2z, a3z")
vBx, vBy = var("vBx, vBy")
aBx, aBy = var("aBx, aBy")
vCx, vCy = var("vCx, vCy")
aCx, aCy = var("aCx, aCy")

sub_list=[
    (a, 1 *m),
    (w1, 2 / s),
]

w1 = Matrix([0,0,-w1])
w2 = Matrix([0,0, w2z])
w3 = Matrix([0,0, w3z])

a1 = Matrix([0, 0, 0])
a2 = Matrix([0,0,a2z])
a3 = Matrix([0,0,a3z])

vA = Matrix([0, 0, 0])
aA = Matrix([0, 0, 0])
vB = Matrix([vBx, vBy, 0])
aB = Matrix([aBx, aBy, 0])
vC = Matrix([vCx, vCy, 0])
aC = Matrix([aCx, aCy, 0])
vD = Matrix([0, 0, 0])
aD = Matrix([0, 0, 0])

dAB = Matrix([a, 0, 0])
h = sqrt( (2*a)**2 - (a/2)**2 )
# h = var("h")
dBC = Matrix([ a/2, h, 0])
dDC = Matrix([-a/2, h, 0])

pprint("\nvB and aB:")
eq1 = Eq( vB, vA + w1.cross(dAB) )
eq2 = Eq( aB, aA + a1.cross(dAB)  + w1.cross(w1.cross(dAB)) )
sol = solve([eq1, eq2], [vBx, vBy, aBx, aBy])
vBx, vBy = sol[vBx], sol[vBy]
aBx, aBy = sol[aBx], sol[aBy]
vB = Matrix([vBx, vBy, 0])
aB = Matrix([aBx, aBy, 0])
pprint(vB)
pprint(aB)

pprint("\nvC aC, w2, w3:")
eq2a = Eq( vC, vB + w2.cross(dBC) )
eq3a = Eq( vC, vD + w3.cross(dDC) )


sol = solve([eq2a, eq3a], [vCx, vCy, w2z, w3z], dict=True)
sol = sol[0]
pprint(sol)
vCx, vCy = sol[vCx], sol[vCy]
w2z, w3z = sol[w2z], sol[w3z]
vC = Matrix([vCx, vCy, 0])
w2 = Matrix([0,0, w2z])
w3 = Matrix([0,0, w3z])

eq2b = Eq( aC, aB + a2.cross(dBC)  + w2.cross(w2.cross(dBC)) )
eq3b = Eq( aC, aD + a3.cross(dDC)  + w3.cross(w3.cross(dDC)) )
sol = solve([eq2b, eq3b], [aCx, aCy, a2z, a3z], dict=True)
sol = sol[0]
pprint(sol)
aCx, aCy = sol[aCx], sol[aCy]
a2z, a3z = sol[a2z], sol[a3z]

aC = Matrix([aCx, aCy, 0])
a2 = Matrix([0,0,a2z])
a3 = Matrix([0,0,a3z])
pprint(vC)
pprint(aC)

pprint(w2)
pprint(w3)
pprint("\na2 and a3:")
pprint(a2)
pprint(a3)

pprint("\nvC / (m/s):")
tmp = vC.subs(sub_list)
tmp /= m/s
tmp = iso_round(tmp,0.01)
pprint(tmp)

# vB and aB:
# ⎡  0  ⎤
# ⎢     ⎥
# ⎢-a⋅ω₁⎥
# ⎢     ⎥
# ⎣  0  ⎦
# ⎡     2⎤
# ⎢-a⋅ω₁ ⎥
# ⎢      ⎥
# ⎢  0   ⎥
# ⎢      ⎥
# ⎣  0   ⎦
#
# vC and aC:
# ⎡-√15⋅a⋅ω₁ ⎤
# ⎢──────────⎥
# ⎢    2     ⎥
# ⎢          ⎥
# ⎢  -a⋅ω₁   ⎥
# ⎢  ──────  ⎥
# ⎢    2     ⎥
# ⎢          ⎥
# ⎣    0     ⎦
# ⎡        2     ⎤
# ⎢   -a⋅ω₁      ⎥
# ⎢   ───────    ⎥
# ⎢      2       ⎥
# ⎢              ⎥
# ⎢            2 ⎥
# ⎢-17⋅√15⋅a⋅ω₁  ⎥
# ⎢──────────────⎥
# ⎢      30      ⎥
# ⎢              ⎥
# ⎣      0       ⎦
#
# w2 and w3:
# ⎡0 ⎤
# ⎢  ⎥
# ⎢0 ⎥
# ⎢  ⎥
# ⎣ω₁⎦
# ⎡0 ⎤
# ⎢  ⎥
# ⎢0 ⎥
# ⎢  ⎥
# ⎣ω₁⎦
#
# a2 and a3:
# ⎡     0     ⎤
# ⎢           ⎥
# ⎢     0     ⎥
# ⎢           ⎥
# ⎢         2 ⎥
# ⎢-2⋅√15⋅ω₁  ⎥
# ⎢───────────⎥
# ⎣     15    ⎦
# ⎡    0    ⎤
# ⎢         ⎥
# ⎢    0    ⎥
# ⎢         ⎥
# ⎢        2⎥
# ⎢2⋅√15⋅ω₁ ⎥
# ⎢─────────⎥
# ⎣    15   ⎦
#
# vC / (m/s):
# ⎡-3.87⎤
# ⎢     ⎥
# ⎢-1.0 ⎥
# ⎢     ⎥
# ⎣ 0.0 ⎦
